## Extending the admin	site with custom views

**views.py**
```python

from django.contrib.admin.views.decorators import staff_member_required
from django.shortcuts import get_object_or_404
from .models import Order

@staff_member_required
def admin_order_detail(request, order_id):
				order =	get_object_or_404(Order, id=order_id)
				return render(request,'admin/orders/order/detail.html',
                               {'order': order})

```

**views.py**

```python
    path('admin/order/<int:order_id>/',	views.admin_order_detail, name='admin_order_detail'),
```

**detail.html**
```python
    extends	"admin/base_site.html"	%}
    {% load	static %}
    {%	block	extrastyle	%}
        <link	rel="stylesheet"	type="text/css"	href="{%	static	"css/admin.css"	%}"	/>
    {%	endblock	%}
    
    {%	block	title	%}
		Order	{{	order.id	}}	{{	block.super	}}
    {%	endblock	%}
    
    {%	block	breadcrumbs	%}
		<div	class="breadcrumbs">
			<a	href="{%	url	"admin:index"	%}">Home</a>	&rsaquo;
			<a	href="{%	url	"admin:orders_order_changelist"	%}">Orders</a>
			<a	href="{%	url	"admin:orders_order_change"	order.id	%}">Order{{	order.id }}</a> Detail
		</div>
    {%	endblock	%}

    {%	block	content	%}
        <h1>Order {{order.id}}</h1>
        <ul	class="object-tools">
            <li>
                <a	href="#"	onclick="window.print();">Print	order</a>
            </li>
        </ul>
        ____
        ____
     {% endblock %}
 ```
    
 
When you want to extend	an	admin template,	you	need to	know its structure	and	identify existing blocks. You can find all	admin templates	at	https://github.com/django/django/tree/2.0/django/contrib/admin/templates/admin

You	can	also	override	an	admin	template	if	you	need	to.	To	override an	admin	template,	copy	it into	your	templates	directory	keeping	the
same relative	path and	filename.	Django's	administration	site	will
use	your	custom	template	instead	of	the	default	one.

**admin.py**
```python
    
    from django.urls import	reverse
    from django.utils.safestring import	mark_safe
    
    def	order_detail(obj):
        return	mark_safe('<a	href="{}">View</a>'.format(
            reverse('orders:admin_order_detail',	args=[obj.id])))	

    class OrderAdmin(admin.ModelAdmin):	
				list_display	=	['id',	
                                     'first_name',	
                                     #	...	
                                     'updated',	
                                     order_detail]
 ```

## Creating pdf

```python
from	django.conf	import	settings
from	django.http	import	HttpResponse
from	django.template.loader	import	render_to_string
import	weasyprint
@staff_member_required
def	admin_order_pdf(request,	order_id):
				order	=	get_object_or_404(Order,	id=order_id)
				html	=	render_to_string('orders/order/pdf.html',
																												{'order':	order})
				response	=	HttpResponse(content_type='application/pdf')
				response['Content-Disposition']	=	'filename=\
								"order_{}.pdf"'.format(order.id)
				weasyprint.HTML(string=html).write_pdf(response,
								stylesheets=[weasyprint.CSS(
												settings.STATIC_ROOT	+	'css/pdf.css')])
				return	response

```
We	use	the	static	file	css/pdf.css	to	add	CSS	styles	to	the
generated	PDF	file.	We	load	it	from	the	local	path	by	using	the
STATIC_ROOT	setting.	Finally,	we	return	the	generated	response.
